In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from operator import itemgetter

In [2]:
path = "C:\\Users\\Tiziano Pacifico\\Desktop\\TESI General\\TESI_FINAL_STAGE\\FASE 6\\DFs\\"
filename = "final_df.json"
df_in = pd.read_json(path+filename, lines=True)

In [3]:
#Divido il dataframe per intervalli settimanali, riduco le colonne a quelle che mi initeressano
df_in['time_utc'] = pd.to_datetime(df_in['utc_datetime_str'])
df_small = df_in[['time_utc','GPT_label','polarity','text']].copy()
day_intervals = pd.date_range(start=df_small['time_utc'].min(), end=df_small['time_utc'].max(), freq='1D')

day_df_list = []
for i in range(len(day_intervals) - 1):
    start_date = day_intervals[i]
    end_date = day_intervals[i + 1]
    week_data = df_small[(df_small['time_utc'] >= start_date) & (df_small['time_utc'] < end_date)].copy()
    day_df_list.append(week_data)
day_df_list.append(df_small[(df_small['time_utc'] >= end_date) & (df_small['time_utc'] <= df_small['time_utc'].max())].copy())

In [4]:
topics = list(df_small['GPT_label'].unique())

In [5]:
#ottengo un dizionario in cui le chiavi sono i topic e i valori sono liste di DFs. Ogni lista contiene i DFs per quei topic divisi per giorni
dfs_per_days_and_topics = {}
for topic in topics:
    days_df_list_per_topic = []
    for dayly_df in day_df_list:
        days_df_list_per_topic.append(dayly_df[dayly_df['GPT_label'] == topic])
    dfs_per_days_and_topics[topic] = days_df_list_per_topic

In [6]:
avg_post_per_days_per_topic_list = {}
for topic in topics:
    avg_post_per_day = (df_small.groupby('GPT_label')['GPT_label'].count()[topic])/len(dfs_per_days_and_topics[topic])
    avg_post_per_days_per_topic_list[topic] = round(avg_post_per_day)

In [7]:
filename = "count_series.json"
count_series = pd.read_json(path+filename)
count_series = count_series.drop('name', axis=1).rename(columns={'index':'topic'})


In [10]:
period_max_activity = {}
i = 0
for topic in topics:
    #print(i)
    
    start = False
    period_max_activity[topic] = []
    start_date = None
    stop_date = None
    j = 0
    for df_per_day_and_topic in dfs_per_days_and_topics[topic]:
        if(df_per_day_and_topic.shape[0] > avg_post_per_days_per_topic_list[topic] and not(start)):
            start = True
            start_date = df_per_day_and_topic['time_utc'].iloc[0]
            stop_date = None
            #print(f"start: {start_date}")
        #print(f"_____{df_per_day_and_topic.shape[0]}")
        if(df_per_day_and_topic.shape[0] < avg_post_per_days_per_topic_list[topic] and start):
            start = False
            if(df_per_day_and_topic.shape[0] <= 0):
                stop_date = day_intervals[j]
                #print(f"stop: {stop_date}")
            else:
                stop_date = df_per_day_and_topic['time_utc'].iloc[-1]
            
        if(start_date != None and stop_date != None):
            period_max_activity[topic].append((start_date, stop_date))
            start_date = None
            stop_date = None
        j += 1
    if(start_date != None and stop_date == None):
        stop_date = dfs_per_days_and_topics[topics[4]][-1]['time_utc'].iloc[-1]
        period_max_activity[topic].append((start_date, stop_date))
    i += 1
#for topic in topics:
#    for p in period_max_activity[topic]:
#        print(p[0],p[1])
#    print("-------------------")

In [11]:
count_series['period_max_Activity'] = count_series['topic'].map(period_max_activity)

In [13]:
period_activity = {}
i = 0
for topic in topics:
    #print(i)
    
    start = False
    period_activity[topic] = []
    start_date = None
    stop_date = None
    j = 0
    for df_per_day_and_topic in dfs_per_days_and_topics[topic]:
        if(df_per_day_and_topic.shape[0] > 0 and not(start)):
            start = True
            start_date = df_per_day_and_topic['time_utc'].iloc[0]
            stop_date = None
            #print(f"start: {start_date}")
        #print(f"_____{df_per_day_and_topic.shape[0]}")
        if(df_per_day_and_topic.shape[0] <= 0 and start):
            start = False
            if(df_per_day_and_topic.shape[0] <= 0):
                stop_date = day_intervals[j]
                #print(f"stop: {stop_date}")
            else:
                stop_date = df_per_day_and_topic['time_utc'].iloc[-1]
            
        if(start_date != None and stop_date != None):
            period_activity[topic].append((start_date, stop_date))
            start_date = None
            stop_date = None
        j += 1
    if(start_date != None and stop_date == None):
        stop_date = dfs_per_days_and_topics[topics[4]][-1]['time_utc'].iloc[-1]
        period_activity[topic].append((start_date, stop_date))
    i += 1

In [15]:
count_series['period_Activity'] = count_series['topic'].map(period_activity)

In [17]:
period_not_max = {}
for topic,periods in period_activity.items():
    period_not_max[topic] = []
    for start,end in periods:
        for start1,end1 in period_max_activity[topic]:
            if (start <= start1 and end >= end1):
                period_not_max[topic].append((start,start1))
                period_not_max[topic].append((end1,end))
            else:
                period_not_max[topic].append((start,end))


In [23]:
df_small.head()

,time_utc,GPT_label,polarity,text,polarity_label
0,2023-05-01 17:00:56,AI Chatbot Interaction and Assistance on Cloud...,0.117045,ChatGPT does not do stand up comedyI don’t thi...,positive
1,2023-05-01 16:59:32,Chatbot Interaction and Usage on Discord,0.158333,Dreamcatcher - ChatGPT Retrieval on SteroidsA ...,positive
2,2023-05-01 16:57:39,Dan Discussing ChatGPT Jailbreaking Ethics,0.166667,"Thoughts on this?[How to Jailbreak ChatGPT, Li...",positive
3,2023-05-01 16:50:02,AI and ChatGPT in Everyday Life,-0.074985,Emergent abilities of chat GPT. We don't reall...,negative
4,2023-05-01 16:43:19,AI Chatbot Interaction and Assistance on Cloud...,0.200000,This chatgpt app gives me a 20 free chat credi...,positive


In [19]:
def polarize(polarity):
    if polarity < 0:
        return 'negative'
    else:
        return 'positive'

In [21]:
df_small['polarity_label'] = df_small['polarity'].apply(polarize)

In [26]:
avg_pos_per_topic_max = {}
for topic,periods in period_max_activity.items():
    df_vuoto = pd.DataFrame(columns=df_small.columns)
    for start,end in periods:
        df_vuoto = pd.concat([df_vuoto, df_small[(df_small['time_utc'] >= start) & (df_small['time_utc'] <= end) & (df_small['GPT_label'] == topic)]])
    avg_pos_per_topic_max[topic] = df_vuoto[df_vuoto['polarity_label'] == 'positive']['polarity'].mean()

In [28]:
avg_neg_per_topic_max = {}
for topic,periods in period_max_activity.items():
    df_vuoto = pd.DataFrame(columns=df_small.columns)
    for start,end in periods:
        df_vuoto = pd.concat([df_vuoto, df_small[(df_small['time_utc'] >= start) & (df_small['time_utc'] <= end) & (df_small['GPT_label'] == topic)]])
    avg_neg_per_topic_max[topic] = df_vuoto[df_vuoto['polarity_label'] == 'negative']['polarity'].mean()

In [29]:
avg_pos_per_topic_not_max = {}
for topic,periods in period_not_max.items():
    df_vuoto = pd.DataFrame(columns=df_small.columns)
    for start,end in periods:
        df_vuoto = pd.concat([df_vuoto, df_small[(df_small['time_utc'] >= start) & (df_small['time_utc'] <= end) & (df_small['GPT_label'] == topic)]])
    avg_pos_per_topic_not_max[topic] = df_vuoto[df_vuoto['polarity_label'] == 'positive']['polarity'].mean()

In [30]:
avg_neg_per_topic_not_max = {}
for topic,periods in period_not_max.items():
    df_vuoto = pd.DataFrame(columns=df_small.columns)
    for start,end in periods:
        df_vuoto = pd.concat([df_vuoto, df_small[(df_small['time_utc'] >= start) & (df_small['time_utc'] <= end) & (df_small['GPT_label'] == topic)]])
    avg_neg_per_topic_not_max[topic] = df_vuoto[df_vuoto['polarity_label'] == 'negative']['polarity'].mean()

In [31]:
count_series['avg_pos_per_topic_max'] = count_series['topic'].map(avg_pos_per_topic_max)
count_series['avg_neg_per_topic_max'] = count_series['topic'].map(avg_neg_per_topic_max)
count_series['avg_pos_per_topic_not_max'] = count_series['topic'].map(avg_pos_per_topic_not_max)
count_series['avg_neg_per_topic_not_max'] = count_series['topic'].map(avg_neg_per_topic_not_max)

In [36]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=list(list(count_series['topic'])),
    x=list(list(count_series['avg_pos_per_topic_max'])),
    name='pos max',
    orientation='h', 
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.add_trace(go.Bar(
    y=list(list(count_series['topic'])),
    x=list(list(count_series['avg_pos_per_topic_not_max'])),
    name='pos not max',
    orientation='h',
    marker=dict(
        color='rgba(58, 71, 80, 0.6)',
        line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
    )
))
fig.update_layout(
    legend=dict(
        title="Legenda",  # Titolo della legenda
        yanchor="top",  # Posizione verticale della legenda
        y=1.0,  # Posizione verticale della legenda
        xanchor="left",  # Posizione orizzontale della legenda
        x=1  # Posizione orizzontale della legenda
    ),
    title=dict(
        text='<b>Confronto tra avg_pos nei periodi di massima attività, rispetto ai periodi di attività regolare',  # Title text
        x=0.5,  # x=0.5 centers the title
        xanchor='center',  # 'center' ensures the title is centered
        font=dict(
            size=18,
            color='black',
            family="Arial, sans-serif",
        )
        )
)


fig.update_layout(
    width=1000,   # Larghezza in pixel
    height=600   # Altezza in pixel
)

In [38]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=list(list(count_series['topic'])),
    x=list(list(count_series['avg_neg_per_topic_max'])),
    name='neg max',
    orientation='h', 
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.add_trace(go.Bar(
    y=list(list(count_series['topic'])),
    x=list(list(count_series['avg_neg_per_topic_not_max'])),
    name='neg not max',
    orientation='h',
    marker=dict(
        color='rgba(58, 71, 80, 0.6)',
        line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
    )
))
fig.update_layout(
    legend=dict(
        title="Legenda",  # Titolo della legenda
        yanchor="top",  # Posizione verticale della legenda
        y=1.0,  # Posizione verticale della legenda
        xanchor="left",  # Posizione orizzontale della legenda
        x=1  # Posizione orizzontale della legenda
    ),
    title=dict(
        text='<b>Confronto tra avg_neg nei periodi di massima attività, rispetto ai periodi di attività regolare',  # Title text
        x=0.5,  # x=0.5 centers the title
        xanchor='center',  # 'center' ensures the title is centered
        font=dict(
            size=18,
            color='black',
            family="Arial, sans-serif",
        )
        )
)


fig.update_layout(
    width=1000,   # Larghezza in pixel
    height=600   # Altezza in pixel
)